## Importation des librairies 


In [1]:
import nltk 
import warnings
from tqdm import tqdm
import pandas as pd
import seaborn as sns
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk.tokenize import sent_tokenize
import re
import gensim
from gensim.utils import simple_preprocess
from gensim import models
from pprint import pprint
import matplotlib.pyplot as plt
import spacy
from spacy.lang.fr.examples import sentences 
nltk.data.path.append("/home/jovyan/teams/syne_data/Commun/NLTK")
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import numpy as np
import gensim.corpora as corpora
from gensim.models import CoherenceModel
from gensim import models 
import collections
from collections import Counter
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
# Ignore warning
import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)
#import sentencepiece
#from numba import jit, njit
import torch  
from scipy.spatial.distance import cosine

In [2]:
#!pip install torch

 ## Importation de la data

In [3]:
data = pd.read_csv("/home/jovyan/workspaces/syne_drh/Data2023/EAC/Entretien_Annuel_de_Competence_2022.csv",delimiter = ';')

FileNotFoundError: [Errno 2] No such file or directory: '/home/jovyan/workspaces/syne_drh/Data2023/EAC/Entretien_Annuel_de_Competence_2022.csv'

In [4]:
data.info()

NameError: name 'data' is not defined

In [39]:
data.shape

(120, 6)

In [40]:
data.head(5)

,library_name,version,bundle_name,support_level,run_deploy,key_words
0,accelerate,0.*,DL,0,0,Python library is designed to optimize and acc...
1,accelerate,0.*,NLP,0,0,Python library is designed to optimize and acc...
2,airflow,2.8.0,AFL,0,0,schedule and monitor workflows
3,anytree,2.*,MIN,1,0,"dataviz to plot decision tree, nodes and links"
4,apache-airflow-providers-vertica,2.*,AFL,0,0,airflow package for vertica provider


In [ ]:
data.isnull().sum()

In [ ]:
print("On a "+str(len(data["Matricule de l'évaluateur"].unique()))+" évaluateurs et "+
str(len(data["Matricule de l'évalué"].unique()))+" évalués")

In [ ]:
def get_most_common_com(colonne,min_count,max_length):
    c = collections.Counter(data[colonne])
    values, counts = zip(*c.most_common())
    most_common = []
    i = 0
    while i<len(values) and (type(values[i])!=str or (counts[i]>min_count and len(values[i])<max_length)) :
        most_common.append(values[i])
        i+=1
    return(most_common)

In [ ]:
get_most_common_com("QVT : Commentaire du collaborateur",1,40)

## Analyse de sentiment pour QVT 

In [ ]:
train = data[["Matricule de l'évalué","Matricule de l'évaluateur","Commentaire général entretien : Collaborateur"]]
train.head()

In [ ]:
train =  train.drop_duplicates(subset=["Matricule de l'évalué"])

In [ ]:
exemple = train.iloc[50]["Commentaire général entretien : Collaborateur"]
print(exemple)
re.split('\n.', exemple)

In [ ]:
A=stopwords.words('french')
A.extend(['X', 'x','xxx','xx','ras','cet', 'ete', 'tres', 'non', 'oui', 'Non', 'Oui', 'a', 'A', 'tout', 'pu', 'faire', 'etre', 'sein',
         'chez', 'lors', 'car', 'va', 'ainsi', 'fait', 'cette', 'afin', 'cela', 'avoir', 'cette', 'doit', 'avant', 'alors', 
         'autre', 'chaque', 'tel', 'tels', 'tellement', 'font', 'bien', 'peu', 'aussi', 'temps', 'tant', 'toute', 'toutes', 'ci', 
         'bon', 'alors', 'au', 'aucuns', 'aussi', 'autre' ,'avant', 'avec' ,'avoir', 'bon' ,'car', 'ce', 'ces', 'ceux', 'chaque',
          'comme',
         'comment', 'dans', 'des', 'du', 'dedans','dehors', 'depuis', 'devrait', 'doit', 'donc', 'dos', 'début', 'elle',
          'elles', 'en', 'encore',
          'essai', 'est', 'et', 'eu',  'fait', 'faites', 'fois', 'font', 'hors', 'ici', 'il', 'ils','ca',
         'je', 'juste' ,'la', 'le', 'les', 'leur', 'là', 'ma' ,'maintenant', 'mais', 'mes', 'mien', 'moins', 'mon', 'mot', 'même','egalement'
          'ni', 'nommés', 'notre' ,'nous', 'ou', 'où', 'par' ,'parce' ,'pas', 'peux', 'cas',
          'peut', 'peu', 'plupart', 'pour', 'pourquoi', 'quand', 'que', 'quel' ,'quelle', 'quelles', 'quels' ,'qui', 'sa', 'sans', 'ses', 'seulement', 'si',
          'sien', 'son', 'ont' ,'sous', 
          'soyez' ,'sujet', 'sur', 'ta', 'tandis', 'tellement' ,'tels', 'tes', 'ton', 'tous', 'tout', 'trop', 'tres', 'tu', 'voient', 'vont', 'votre', 'vous',
         'vu', 'ça', 'étaient', 'etat', 'étions', 'ete', 'etre', 'entre', 'beaucoup', 'jour', 'annees','annee' 'plus', 'trouve', 'meme', 'dont', 'vue', 'nan','toujours','concernant','plus'])
  
def enleve_stop_words(text):
    words = text.split(" ")
    words_net= [item for item in words if item not in A] 
    return(" ".join(words_net))

def preprocessing(text,stop_words = True):
    if isinstance(text, str):
        text = str(text)
        #lowercase
        text = text.lower()
        #accents and punctuations
        text = re.sub("\d+", " ", text) #numbers normalization
        if stop_words:
            text = enleve_stop_words(text)
        text = re.sub('[éèê]', "e", text) #accents removal
        text = re.sub("[.,;:!?]", " ", text)
        text = re.sub("[|{}\[\]()«»/]", " ", text)
        text = re.sub("[“”]", " ", text)
        text = re.sub("'", " ", text) 
        text = re.sub("", " ", text) 
        text = re.sub('"', " ", text)
        text = re.sub('[+-]', " ", text)
        text = re.sub('[=*/]', " ", text)
        text = re.sub("ô", "o", text)
        text = re.sub("°", "", text)
        text = re.sub("\x80", "", text)
        text = re.sub("[«»€$%;:.,?!=()+&@*%^¨~#><]{1,}", " ", text)
        text = re.sub("[\/]{1,}", " ", text)
        text = re.sub("'", " ", text)
        text = re.sub('"', " ", text)
        #Symbols
        text = re.sub("[€%$£]", "", text)
        #End of lines and parahraph breaks
        text = re.sub('\r\n', " ", text)
        text = re.sub('\n', " ", text) #end of line removal
        #white spaces
        text = re.sub('\s+', " ", text) #white space removal
        text = text.rstrip(" ") #à droite
        text = text.lstrip(" ") #à gauche
        if stop_words:
            text = enleve_stop_words(text)
    else:
        return('')
    return  text

In [ ]:
#train["QVT : Commentaire du collaborateur"] = train["QVT : Commentaire du collaborateur"].apply(lambda x:preprocessing(x,stop_words=False))
train["clean_commentaire"] = train["Commentaire général entretien : Collaborateur"].apply(lambda x:preprocessing(x))

In [ ]:
def get_most_common_com(colonne,min_count,max_length):
    c = collections.Counter(train[colonne])
    values, counts = zip(*c.most_common())
    most_common = []
    i = 0
    while i<len(values) and (type(values[i])!=str or (counts[i]>min_count and len(values[i])<max_length)) :
        most_common.append(values[i])
        i+=1
    return(most_common)

In [ ]:
get_most_common_com("clean_commentaire",1,100)

In [ ]:
train = train[(train["clean_commentaire"] != '') &
          
          (train["clean_commentaire"] != 'rien ajouter') ]

In [ ]:
train['temp_list'] = train["clean_commentaire"].apply(lambda x:enleve_stop_words(x)).apply(lambda x:str(x).split())
mots_insignifactif = ["entretien","annee","mission","echange"]
top = Counter([item for sublist in train['temp_list'] for item in sublist if item not in mots_insignifactif ])
temp = pd.DataFrame(top.most_common(20))
temp.columns = ['Common_words','count']
temp

In [ ]:
from plotly import graph_objs as go
import plotly.express as px
fig = px.bar(temp, x="count", y="Common_words", title='Commmon Words in Selected Text', orientation='h', 
             width=700, height=700,color='Common_words')
fig.show()

In [ ]:
fig = px.treemap(temp, path=['Common_words'], values='count',title='Tree of Most Common Words')
fig.show()

In [ ]:
wordcloud = WordCloud(width=800, height=400, background_color='white').generate_from_frequencies(top)
# Afficher le nuage de mots
plt.figure(figsize=(10, 5))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.show()

In [ ]:
nlp = spacy.load("fr_core_news_sm", disable=["parser", "ner"])

In [ ]:
def get_emmebding(sentence):
    tokenized_sentence = tokenizer.tokenize(sentence)
    encoded_sentence = tokenizer.encode(tokenized_sentence)
    encoded_sentence = torch.tensor(encoded_sentence).unsqueeze(0)
    embeddings = camembert(encoded_sentence).last_hidden_state.mean(dim=1)
    return(embeddings[0].detach().numpy())

In [ ]:
questions = ["que est ce que tu penses de charge de travail ?",
             "que est ce que tu penses de l'organisation du travail ?",
             "tu peux équilibrer entre vie privee et vie professionnelle ? ",
             "que est ce que tu penses de la déconnexion de travail?",
             "que est ce que tu penses de ton salaire?"]

emmebding_themes = [get_emmebding(questions[i])  for i in range(5)]
def get_similarity(sentence):
    emmebding = get_emmebding(sentence)
    similarities = [1-cosine(emmebding_themes[i], emmebding) for i in range(5)]
    scaler = MinMaxScaler()
    scaled_probabilities = scaler.fit_transform(np.array(similarities).reshape(-1,1))
    return(scaled_probabilities)

In [ ]:
sentence ="L'organisation au sein de l'equipe est rodee, on nous laisse une autonomie et travailler en toute autonomie avec un soutien de managers si besoin.Je me suis adaptee au rythme du teletravail"
get_similarity(sentence)

In [ ]:
 
# Définir la fonction pour extraire les mots avant "permet"
def extraire_mots_avant_permet(liste_de_mots):
    mots_avant = []
    for i in range(len(liste_de_mots)):
        if i >= 1 and liste_de_mots[i] == 'entretien':
            mots_avant.append(liste_de_mots[i - 1])
    return mots_avant

# Appliquer la fonction aux commentaires
liste_mots_avant = train["temp_list"].apply(extraire_mots_avant_permet)

# Compter le nombre d'occurrences de chaque mot
compteur_mots_avant = Counter([mot for liste in liste_mots_avant for mot in liste])
temp = pd.DataFrame(compteur_mots_avant.most_common(10))
temp.columns = ['Common_words','count']
# Créer le premier graphique
fig1 = px.bar(temp, x="count", y="Common_words", title='Mots avant "entretien"', orientation='h',
              width=600, height=400, color='Common_words')

# Définir la fonction pour extraire les mots après "permet"
def extraire_mots_apres_permet(liste_de_mots):
    mots_apres = []
    for i in range(len(liste_de_mots)):
        if i < len(liste_de_mots) - 1 and liste_de_mots[i] == 'entretien':
            mots_apres.append(liste_de_mots[i + 1])
    return mots_apres

# Appliquer la fonction aux commentaires
liste_mots_apres = train["temp_list"].apply(extraire_mots_apres_permet)

# Compter le nombre d'occurrences de chaque mot
compteur_mots_apres = Counter([mot for liste in liste_mots_apres for mot in liste])
temp = pd.DataFrame(compteur_mots_apres.most_common(10))
temp.columns = ['Common_words','count']
# Créer le deuxième graphique
fig2 = px.bar(temp, x="count", y="Common_words", title='Mots après "permet"', orientation='h',
              width=600, height=400, color='Common_words')

# Créer un subplot avec les deux graphiques côte à côte
from IPython.display import HTML
fig1_html = fig1.to_html(full_html=False, default_height=400, default_width=450)
fig2_html = fig2.to_html(full_html=False, default_height=400, default_width=450)
HTML(f'<table><tr><td>{fig1_html}</td><td>{fig2_html}</td></tr></table>')


## Tester les modèles

In [ ]:
def preprocessing(text):
    try:
        text = text.lower()
        #text = re.sub('[éèê]', "e", text) #accents removal
        #Symbols
        text = re.sub("[€%$£]", "", text)
        #End of lines and parahraph breaks
        text = text.rstrip(" ") #à droite
        text = text.lstrip(" ") #à gauche
        text = re.sub('\s+', " ", text)
        A = stopwords.words('french')
        words = text.split(" ")
        words_net= [item for item in words if item not in A] 
        text = " ".join(words_net)
        return(text)
    except:
        return('')

In [ ]:
sample_test = pd.read_csv("data/Echantillonnage_supérvisé.csv", sep = ";",encoding='ISO-8859-1')
sample_test = sample_test[["Matricule de l'Ã©valuÃ©","Commentaire gÃ©nÃ©ral entretien : Collaborateur","Vrai score entretien"]]
sample_test

In [ ]:
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification, pipeline

tokenizerRRR= AutoTokenizer.from_pretrained("/home/jovyan/teams/syne_data/Commun/sentiment_analysis_transformer/models/tokenizer/")

modelLLL = TFAutoModelForSequenceClassification.from_pretrained("/home/jovyan/teams/syne_data/Commun/sentiment_analysis_transformer/models/TFAutoModelForSequenceClassification/") 

nlpYOU_1= pipeline('sentiment-analysis', model=modelLLL, tokenizer=tokenizerRRR)

In [ ]:
def get_label_score_1(text):
    try:
        result = nlpYOU_1(text)[0]
        label = result.get("label")
        score = result.get("score")
        txt_return = label + ":" + str(score)
        return txt_return
    except:
        return "erreur: "

In [ ]:
sample_test = pd.read_csv("data/Echantillonnage_supérvisé.csv", sep = ";",encoding='ISO-8859-1')

In [ ]:
sample_test = sample_test[["Matricule de l'Ã©valuÃ©","QVT : Commentaire du collaborateur",'Vrai score QRV']]
sample_test

In [ ]:
sample = train[train["Matricule de l'évalué"].isin(sample_test["Matricule de l'Ã©valuÃ©"])]
sample = sample.drop("temp_list",axis=1)
sample

In [ ]:
%%time
sample["score"] = sample["Commentaire général entretien : Collaborateur"].apply(lambda x : get_label_score_1(preprocessing(x)))
sample[['labl','score']]= sample.score.str.split(':',expand=True)

In [ ]:
sample.head(3)

In [ ]:
sample["true score"] = sample["Matricule de l'évalué"].apply(lambda x:sample_test[sample_test["Matricule de l'Ã©valuÃ©"]==x]["Vrai score QRV"].values[0])

In [ ]:
sample

In [ ]:
sample['score'] = pd.to_numeric(sample['score'], errors='coerce')

In [ ]:
sample['level'] = pd.qcut(sample['score'],5, labels=[1, 2, 3,4,5])
score = sample["true score"].values
level = sample["level"].values
c = 0 
for i in range(len(score)):
    if np.abs(score[i]-level[i])<=1:
        c+=1
print(c/len(score))

from sklearn.metrics import confusion_matrix
confusion = confusion_matrix(score,level)
sns.heatmap(confusion, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Prédictions')
plt.ylabel('Vraies étiquettes')
plt.title('Matrice de Confusion pour le commentaire de colaborateur',fontsize=10)

### Tester modèle 2

In [ ]:
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification, pipeline
tokenizerRRR= AutoTokenizer.from_pretrained("models/sentiment_analysis" )
modelLLL = TFAutoModelForSequenceClassification.from_pretrained("models/sentiment_analysis") 

In [ ]:
nlpYOU = pipeline("text-classification",model=modelLLL, tokenizer=tokenizerRRR)

In [ ]:
import tensorflow as tf
def get_proba(text):
    inputs= tokenizerRRR(text,return_tensors="tf")
    outputs = modelLLL(**inputs)
    logits = outputs.logits
    return(np.array(tf.nn.softmax(logits))[0])

In [ ]:
print(get_proba("Alad'2 est clairement le meilleur film de l'année 2018.")) # POSITIVE

In [ ]:
print(nlpYOU("Alad'2 est clairement le meilleur film de l'année 2018.")) # POSITIVE

In [ ]:
def get_score(text):
    if text == '':
        return(None)
    else : 
        t = nlpYOU(text)
        return(int(t[0]['label'][0]))
    #t = get_proba(text)
    #return(sum([t[i]*(i+1) for i in range(5)]))

In [ ]:
sample

In [ ]:
level = sample["Commentaire général entretien : Collaborateur"].apply(lambda text:get_score(preprocessing(text))).values
score = sample["true score"].values

In [ ]:
c = 0 
for i in range(len(score)):
    if np.abs(score[i]-level[i])<=1:
        c+=1
print(c/len(score))

In [ ]:
confusion = confusion_matrix(score,level)
sns.heatmap(confusion, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Prédictions')
plt.ylabel('Vraies étiquettes')
plt.title('Matrice de Confusion pour le commentaire de colaborateur',fontsize=10)

### tester modèle 3

In [ ]:
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification, pipeline
tokenizerRRR= AutoTokenizer.from_pretrained("models/cmakea" )
modelLLL = TFAutoModelForSequenceClassification.from_pretrained("models/cmakea") 

In [ ]:
nlpYOU = pipeline("text-classification",model=modelLLL, tokenizer=tokenizerRRR)

In [ ]:
print(nlpYOU("Alad'2 est clairement le meilleur film de l'année 2018.")) # POSITIVE

In [ ]:
def get_score(text):
    if text == '':
        return(None)
    else : 
        t = nlpYOU(text)
        return(int(t[0]['label'][0]))
    #t = get_proba(text)
    #return(sum([t[i]*(i+1) for i in range(5)]))

In [ ]:
level = sample["Commentaire général entretien : Collaborateur"].apply(lambda text:get_score(preprocessing(text))).values
score = sample["true score"].values

In [ ]:
c = 0 
for i in range(len(score)):
    if np.abs(score[i]-level[i])<=1:
        c+=1
print(c/len(score))

In [ ]:
confusion = confusion_matrix(score,level)
sns.heatmap(confusion, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Prédictions')
plt.ylabel('Vraies étiquettes')
plt.title('Matrice de Confusion pour le commentaire de colaborateur',fontsize=10)

## Analyse de sentiment

In [ ]:
train.head(3)

In [ ]:
train['general_entretien_collaborateur_score'] = train['Commentaire général entretien : Collaborateur'].apply(lambda text : get_score(preprocessing(text)))

In [ ]:
train.head(3)

In [ ]:
train[["Matricule de l'évalué","Matricule de l'évaluateur","Commentaire général entretien : Collaborateur","general_entretien_collaborateur_score"]].to_csv('data/sentiment_analyse_entretien_collaborateur.csv',index=False)

## Analyse de résultats

In [ ]:
df = pd.read_csv('data/sentiment_analyse_entretien_collaborateur.csv')

In [ ]:
df.info()

In [ ]:
df.head(3)

In [ ]:
sns.countplot(data=df, x="general_entretien_collaborateur_score")
plt.title(f'general_entretien_collaborateur_score')
plt.xlabel(f'Score')
plt.ylabel("Nombre d'occurrences")
 
# Afficher le graphique
plt.show()

In [ ]:
A=stopwords.words('french')
A.extend(['X', 'x','xxx','xx','ras','cet', 'ete', 'tres', 'non', 'oui', 'Non', 'Oui', 'a', 'A', 'tout', 'pu', 'faire', 'etre', 'sein',
         'chez', 'lors', 'car', 'va', 'ainsi', 'fait', 'cette', 'afin', 'cela', 'avoir', 'cette', 'doit', 'avant', 'alors', 
         'autre', 'chaque', 'tel', 'tels', 'tellement', 'font', 'bien', 'peu', 'aussi', 'temps', 'tant', 'toute', 'toutes', 'ci', 
         'bon', 'alors', 'au', 'aucuns', 'aussi', 'autre' ,'avant', 'avec' ,'avoir', 'bon' ,'car', 'ce', 'ces', 'ceux', 'chaque',
          'comme',
         'comment', 'dans', 'des', 'du', 'dedans','dehors', 'depuis', 'devrait', 'doit', 'donc', 'dos', 'début', 'elle',
          'elles', 'en', 'encore',
          'essai', 'est', 'et', 'eu',  'fait', 'faites', 'fois', 'font', 'hors', 'ici', 'il', 'ils','ca',
         'je', 'juste' ,'la', 'le', 'les', 'leur', 'là', 'ma' ,'maintenant', 'mais', 'mes', 'mien', 'moins', 'mon', 'mot', 'même','egalement'
          'ni', 'nommés', 'notre' ,'nous', 'ou', 'où', 'par' ,'parce' ,'pas', 'peux', 'cas',
          'peut', 'peu', 'plupart', 'pour', 'pourquoi', 'quand', 'que', 'quel' ,'quelle', 'quelles', 'quels' ,'qui', 'sa', 'sans', 'ses', 'seulement', 'si',
          'sien', 'son', 'ont' ,'sous', 
          'soyez' ,'sujet', 'sur', 'ta', 'tandis', 'tellement' ,'tels', 'tes', 'ton', 'tous', 'tout', 'trop', 'tres', 'tu', 'voient', 'vont', 'votre', 'vous',
         'vu', 'ça', 'étaient', 'etat', 'étions', 'ete', 'etre', 'entre', 'beaucoup', 'jour', 'annees','annee' 'plus', 'trouve', 'meme', 'dont', 'vue', 'nan','toujours','concernant','plus'])
  
def enleve_stop_words(text):
    words = text.split(" ")
    words_net= [item for item in words if item not in A] 
    return(" ".join(words_net))

def preprocessing(text,stop_words = True):
    if isinstance(text, str):
        text = str(text)
        #lowercase
        text = text.lower()
        #accents and punctuations
        text = re.sub("\d+", " ", text) #numbers normalization
        if stop_words:
            text = enleve_stop_words(text)
        text = re.sub('[éèê]', "e", text) #accents removal
        text = re.sub("[.,;:!?]", " ", text)
        text = re.sub("[|{}\[\]()«»/]", " ", text)
        text = re.sub("[“”]", " ", text)
        text = re.sub("'", " ", text) 
        text = re.sub("", " ", text) 
        text = re.sub('"', " ", text)
        text = re.sub('[+-]', " ", text)
        text = re.sub('[=*/]', " ", text)
        text = re.sub("ô", "o", text)
        text = re.sub("°", "", text)
        text = re.sub("\x80", "", text)
        text = re.sub("[«»€$%;:.,?!=()+&@*%^¨~#><]{1,}", " ", text)
        text = re.sub("[\/]{1,}", " ", text)
        text = re.sub("'", " ", text)
        text = re.sub('"', " ", text)
        #Symbols
        text = re.sub("[€%$£]", "", text)
        #End of lines and parahraph breaks
        text = re.sub('\r\n', " ", text)
        text = re.sub('\n', " ", text) #end of line removal
        #white spaces
        text = re.sub('\s+', " ", text) #white space removal
        text = text.rstrip(" ") #à droite
        text = text.lstrip(" ") #à gauche
        if stop_words:
            text = enleve_stop_words(text)
    else:
        return('')
    return  text

In [ ]:
for i in range(1,6):
    Deconnexion_pro_perso= df["Commentaire général entretien : Collaborateur"][df["general_entretien_collaborateur_score"]==i].apply(lambda x:preprocessing(x)).apply(lambda x:str(x).split())
    top = Counter([item for sublist in Deconnexion_pro_perso for item in sublist ])
    temp = pd.DataFrame(top.most_common(10))
    temp.columns = ['Common_words','count']
    mots_a_supprimer = ["entretien","annee","echange","competence","mission"]

    top = Counter({mot: top[mot] for mot in  top if mot not in mots_a_supprimer})

    #fig = px.treemap(temp, path=['Common_words'], values='count',title='Tree of Most Common Words in Deconnexion/articulation column')
    #fig.show()
    wordcloud = WordCloud(width=800, height=400, background_color='white').generate_from_frequencies(top)

    # Afficher le nuage de mots
    plt.figure(figsize=(10, 5))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.title("Score "+str(i))
    plt.axis('off')
    plt.show()

In [34]:
#data=pd.read_csv('supported_python_librairies.csv',sep=',')

In [1]:
pip list

Package                       Version
----------------------------- --------------------
alabaster                     0.7.12
anaconda-client               1.11.0
anaconda-navigator            2.3.2
anaconda-project              0.11.1
annotated-types               0.7.0
anyio                         3.5.0
appdirs                       1.4.4
argon2-cffi                   21.3.0
argon2-cffi-bindings          21.2.0
arrow                         1.2.2
asgiref                       3.6.0
astroid                       2.11.7
astropy                       5.1
atomicwrites                  1.4.0
attrs                         21.4.0
Automat                       20.2.0
autopep8                      1.6.0
Babel                         2.9.1
backcall                      0.2.0
backports.functools-lru-cache 1.6.4
backports.tempfile            1.0
backports.weakref             1.0.post1
bcrypt                        3.2.0
beautifulsoup4                4.11.1
binaryornot                   0.4.4
bi

In [1]:
!pip install transformers

Defaulting to user installation because normal site-packages is not writeable
     ---------------------------------------- 9.4/9.4 MB 6.5 MB/s eta 0:00:00
     ---------------------------------------- 2.2/2.2 MB 8.3 MB/s eta 0:00:00
     -------------------------------------- 287.9/287.9 kB 6.0 MB/s eta 0:00:00
     -------------------------------------- 417.5/417.5 kB 8.9 MB/s eta 0:00:00
     -------------------------------------- 177.6/177.6 kB 5.4 MB/s eta 0:00:00


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [2]:
# Différents imports



from transformers import pipeline

import pandas as pd



import warnings

warnings.filterwarnings('ignore')



In [3]:
# Initialisation du texte à traiter



text = """I ordered a used book from your online site. \

The book is Les mystères de Paris from Victor Hugo. \

It was indicated on the site that it was in good condition. \

I received it a week after ordering. \

And while unpacking the package I realized that it was damaged (damaged pages, writings). \

I contacted customer service who proceeded to the immediate refund."""



In [4]:
# Traduction



translator = pipeline('translation_en_to_fr')

outputs = translator(text, clean_up_tokenization_spaces=True, min_length=100)

print(outputs[0]['translation_text'])

No model was supplied, defaulted to google-t5/t5-base and revision 686f1db (https://huggingface.co/google-t5/t5-base).
Using a pipeline without specifying a model name and revision in production is not recommended.


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

J'ai commandé un livre d'occasion à partir de votre site en ligne. Le livre est Les mystères de Paris de Victor Hugo. Il était indiqué sur le site qu'il était en bon état. J'ai reçu le livre une semaine après avoir commandé et alors que je déballais le paquet, je me suis rendu compte qu'il était endommagé (pages endommagées, écrits).


In [5]:
# Text Classification



classifier = pipeline("text-classification")

outputs = classifier(text)

pd.DataFrame(outputs)


No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

,label,score
0,NEGATIVE,0.998112


In [6]:
# Name Entity Recognition



ner_tagger = pipeline("ner", aggregation_strategy="simple")

outputs = ner_tagger(text)

pd.DataFrame(outputs)

No model was supplied, defaulted to dbmdz/bert-large-cased-finetuned-conll03-english and revision f2482bf (https://huggingface.co/dbmdz/bert-large-cased-finetuned-conll03-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


config.json:   0%|          | 0.00/998 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.33G [00:00<?, ?B/s]

Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/60.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

,entity_group,score,word,start,end
0,MISC,0.978184,Les mystères de Paris,58,79
1,PER,0.900909,Victor Hugo,85,96


In [7]:
ner_tagger = pipeline("ner", aggregation_strategy="simple")

outputs = ner_tagger("Donald Trump, the former president, is now playing golf all days in Florida with his friend Bill")

pd.DataFrame(outputs)

No model was supplied, defaulted to dbmdz/bert-large-cased-finetuned-conll03-english and revision f2482bf (https://huggingface.co/dbmdz/bert-large-cased-finetuned-conll03-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


,entity_group,score,word,start,end
0,PER,0.999558,Donald Trump,0,12
1,LOC,0.999614,Florida,68,75
2,PER,0.999508,Bill,92,96


In [12]:
# Question Answering



reader = pipeline("question-answering")

results = []

questions = ["What was ordered?",

"What did the customer service?"]

for question in questions:

outputs = reader(question=question, context=text)

results.append(outputs)



pd.DataFrame(results)

IndentationError: expected an indented block (2237135414.py, line 15)

In [9]:
# Summarization



summarizer = pipeline("summarization")

outputs = summarizer(text, max_length=150, clean_up_tokenization_spaces=True)

print(outputs[0]['summary_text'])





No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.


config.json:   0%|          | 0.00/1.80k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

Your max_length is set to 150, but your input_length is only 86. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=43)


 The book is Les mystères de Paris from Victor Hugo. It was indicated on the site that it was in good condition. But while unpacking the package I realized it was damaged (damaged pages, writings) I contacted customer service who proceeded to the immediate refund.


In [10]:
# Text Generation



generator = pipeline("text-generation")

response = "Dear friends. Let me tell you my last experience with this online store where I ordered a book."

prompt = text + "\n\nStory I told to my friends:\n\n" + response

outputs = generator(prompt, max_length=200)

print(outputs[0]['generated_text'])

No model was supplied, defaulted to openai-community/gpt2 and revision 6c0e608 (https://huggingface.co/openai-community/gpt2).
Using a pipeline without specifying a model name and revision in production is not recommended.


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


I ordered a used book from your online site. 
The book is Les mystères de Paris from Victor Hugo. 
It was indicated on the site that it was in good condition. 
I received it a week after ordering. 
And while unpacking the package I realized that it was damaged (damaged pages, writings). 
I contacted customer service who proceeded to the immediate refund.

Story I told to my friends:

Dear friends. Let me tell you my last experience with this online store where I ordered a book. I ordered it on my computer a week before Christmas. 

It was in good condition but the title, it contains several books containing a very large amount of material with some essays and many short passages. 

What could say this guy should know better?

And my question is: What did I bring with me here for a purchase, that we can understand? 

And what happened to them


In [1]:
["qualité de vie au travail"
"bien-être"
"équilibre vie pro/vie perso"
"stress"
"charge de travail"
"environnement de travail"
"flexibilité"
"conditions de travail"]

['qualité de vie au travailbien-êtreéquilibre vie pro/vie persostresscharge de travailenvironnement de travailflexibilitéconditions de travail']

In [3]:
import pandas as pd
import re

# Exemples de commentaires dans la colonne 'commentaires'
data = {'commentaires': [
    "Je trouve que la qualité de vie au travail est très bonne, mais la rémunération est trop faible.",
    "L'équilibre entre vie professionnelle et personnelle est bon, mais le stress est élevé.",
    "Les conditions de travail sont stressantes et la charge de travail est trop importante.",
    "La rémunération est correcte, mais je manque de flexibilité dans mon emploi du temps."
]}

# Création d'un DataFrame
df = pd.DataFrame(data)
df

,commentaires
0,Je trouve que la qualité de vie au travail est...
1,L'équilibre entre vie professionnelle et perso...
2,Les conditions de travail sont stressantes et ...
3,"La rémunération est correcte, mais je manque d..."


In [4]:
# Liste de mots-clés associés à la qualité de vie au travail
qvt_keywords = [
    r'\bqualité de vie au travail\b', r'\bbien-être\b', r'\béquilibre\b', 
    r'\bstress\b', r'\bcharge de travail\b', r'\benvironnement de travail\b',
    r'\bflexibilité\b', r'\bconditions de travail\b'
]

In [5]:
# Fonction pour extraire les parties du commentaire qui contiennent des mots-clés QVT
def extract_qvt(comment):
    # Regrouper tous les mots-clés sous une seule expression régulière
    pattern = '|'.join(qvt_keywords)
    # Trouver toutes les correspondances de QVT dans le commentaire
    matches = re.findall(pattern, comment, flags=re.IGNORECASE)
    if matches:
        return comment  # Renvoie tout le commentaire si un mot-clé est trouvé
    return None

In [7]:
# Appliquer l'extraction sur la colonne 'commentaires'
df['commentaires_QVT'] = df['commentaires'].apply(extract_qvt)

In [8]:
print(df[['commentaires', 'commentaires_QVT']])

                                        commentaires  \
0  Je trouve que la qualité de vie au travail est...   
1  L'équilibre entre vie professionnelle et perso...   
2  Les conditions de travail sont stressantes et ...   
3  La rémunération est correcte, mais je manque d...   

                                    commentaires_QVT  
0  Je trouve que la qualité de vie au travail est...  
1  L'équilibre entre vie professionnelle et perso...  
2  Les conditions de travail sont stressantes et ...  
3  La rémunération est correcte, mais je manque d...  


In [12]:
#Si tu veux extraire uniquement une portion spécifique du commentaire (autour du mot-clé), tu peux adapter l'extraction comme suit :
#
def extract_qvt_part(comment):
    # Regrouper tous les mots-clés sous une seule expression régulière
    pattern = '|'.join(qvt_keywords)
    # Trouver le premier match et extraire une portion autour du mot-clé
    match = re.search(pattern, comment, flags=re.IGNORECASE)
    if match:
        start = max(0, match.start() - 50)  # Extraire 50 caractères avant le mot-clé
        end = min(len(comment), match.end() + 50)  # Extraire 50 caractères après le mot-clé
        return comment[start:end]
    return None

# Appliquer l'extraction sur la colonne 'commentaires'
df['commentaires_QVT_part'] = df['commentaires'].apply(extract_qvt_part)

# Affichage des résultats
print(df[['commentaires', 'commentaires_QVT_part']])


                                        commentaires  \
0  Je trouve que la qualité de vie au travail est...   
1  L'équilibre entre vie professionnelle et perso...   
2  Les conditions de travail sont stressantes et ...   
3  La rémunération est correcte, mais je manque d...   

                               commentaires_QVT_part  
0  Je trouve que la qualité de vie au travail est...  
1  L'équilibre entre vie professionnelle et perso...  
2  Les conditions de travail sont stressantes et ...  
3  La rémunération est correcte, mais je manque d...  


In [13]:
import pandas as pd

# Exemple de DataFrame avec des commentaires séparés par une barre verticale '|'
data = {'commentaires': [
    "Équilibre vie pro/vie perso: bon | Qualité de vie au travail: satisfaisante | Rémunération: moyenne",
    "Équilibre: difficile | QVT: stressant | Rémunération: faible",
    "Vie personnelle impactée | Bien-être au travail: excellent | Salaire correct",
    "Équilibre correct | Conditions de travail médiocres | Salaire insuffisant"
]}

# Création d'un DataFrame
df = pd.DataFrame(data)

# Séparateur utilisé dans les commentaires (ici la barre verticale '|')
separateur = '|'

# Séparer chaque commentaire en fonction du séparateur
df['commentaire_parts'] = df['commentaires'].str.split(separateur)

# Supposons que la qualité de vie au travail soit toujours la deuxième partie du commentaire (index 1)
df['commentaires_QVT'] = df['commentaire_parts'].apply(lambda x: x[1].strip() if len(x) > 1 else None)

# Afficher les résultats
print(df[['commentaires', 'commentaires_QVT']])


                                        commentaires  \
0  Équilibre vie pro/vie perso: bon | Qualité de ...   
1  Équilibre: difficile | QVT: stressant | Rémuné...   
2  Vie personnelle impactée | Bien-être au travai...   
3  Équilibre correct | Conditions de travail médi...   

                           commentaires_QVT  
0  Qualité de vie au travail: satisfaisante  
1                            QVT: stressant  
2           Bien-être au travail: excellent  
3           Conditions de travail médiocres  


In [14]:
import re

# Liste de mots-clés associés à la qualité de vie au travail
qvt_keywords = [
    r'qualité de vie au travail', r'QVT', r'bien-être', r'conditions de travail'
]

# Fonction pour trouver la partie QVT après avoir séparé le commentaire
def find_qvt_part(parts):
    pattern = '|'.join(qvt_keywords)  # Fusionner les mots-clés en une seule expression régulière
    for part in parts:
        if re.search(pattern, part, flags=re.IGNORECASE):
            return part.strip()  # Retourner la partie contenant un mot-clé
    return None

# Appliquer la fonction sur les parties du commentaire
df['commentaires_QVT'] = df['commentaire_parts'].apply(find_qvt_part)

# Afficher les résultats
print(df[['commentaires', 'commentaires_QVT']])


                                        commentaires  \
0  Équilibre vie pro/vie perso: bon | Qualité de ...   
1  Équilibre: difficile | QVT: stressant | Rémuné...   
2  Vie personnelle impactée | Bien-être au travai...   
3  Équilibre correct | Conditions de travail médi...   

                           commentaires_QVT  
0  Qualité de vie au travail: satisfaisante  
1                            QVT: stressant  
2           Bien-être au travail: excellent  
3           Conditions de travail médiocres  


In [17]:
#from transformers import pipeline

# Charger un pipeline pour l'analyse de sentiment
#sentiment_pipeline = pipeline("sentiment-analysis")

# Analyse de sentiment d'une phrase
#result = sentiment_pipeline("I love using Hugging Face models!")
#print(result)


In [ ]:
from flair.models import TextClassifier
from flair.data import Sentence
from textblob import TextBlob

# Charger le modèle pré-entraîné
classifier = TextClassifier.load('en-sentiment')

# Analyse de sentiment
sentence = Sentence("Flair is pretty neat for sentiment analysis.")
classifier.predict(sentence)

# Afficher le résultat
print(sentence.labels)

from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

# Initialiser l'analyseur
analyzer = SentimentIntensityAnalyzer()

# Analyse de sentiment
result = analyzer.polarity_scores("I love natural language processing!")
print(result)

import spacy
from textblob import TextBlob

nlp = spacy.load('en_core_web_sm')

# Analyse de texte
doc = nlp("I am very happy with the results of my analysis!")

# Utilisation de TextBlob pour l'analyse de sentiment
sentiment = TextBlob(doc.text).sentiment
print(sentiment)


Pour effectuer de l'analyse de sentiment, vous avez à disposition plusieurs modèles et bibliothèques. Si vous voulez des performances optimales, les modèles basés sur Transformers comme BERT ou RoBERTa sont idéaux. Pour des solutions plus simples et rapides à mettre en place, vous pouvez vous tourner vers des bibliothèques comme Flair, TextBlob, ou VADER. Le choix du modèle ou de la bibliothèque dépend de votre cas d'usage, de la langue et de la taille des données disponibles.


GloVe, Word2Vec, et les embeddings (ou word embeddings) sont des techniques populaires en NLP (Natural Language Processing) pour représenter des mots sous forme de vecteurs numériques. Elles ont été utilisées avant l'émergence des modèles basés sur transformers comme BERT, mais elles sont toujours très utiles dans certaines applications. Voici une explication détaillée de chaque méthode et leur rôle dans des tâches comme l'analyse de sentiment.

Applications en analyse de sentiment :

Word2Vec produit des vecteurs de mots qui peuvent être utilisés comme entrée dans des modèles de machine learning pour des tâches comme l'analyse de sentiment.
Cependant, Word2Vec génère des représentations statiques des mots : un mot a toujours la même représentation, quel que soit le contexte (ce qui est une limitation par rapport aux modèles comme BERT qui produisent des représentations contextuelles).
Utilisation : Vous pouvez utiliser la bibliothèque Gensim pour charger des modèles Word2Vec pré-entraînés ou entraîner votre propre modèle.

GloVe produit des embeddings statiques comme Word2Vec, mais il est particulièrement efficace pour capturer des relations sémantiques entre les mots.
Il est souvent utilisé avec des modèles de machine learning comme les LSTM ou les réseaux de neurones pour des tâches de NLP.

Conclusion
Word2Vec et GloVe sont des techniques classiques de word embeddings qui produisent des représentations statiques des mots. Elles sont simples à utiliser et efficaces dans de nombreuses tâches de NLP, y compris l'analyse de sentiment.
Les transformers comme BERT produisent des embeddings contextuels et surpassent généralement les techniques statiques dans des tâches comme l'analyse de sentiment, car ils prennent en compte le contexte complet dans lequel un mot apparaît.
Embeddings comme ceux générés par Word2Vec et GloVe restent utiles dans des architectures plus simples ou lorsque les ressources de calcul sont limitées.

In [ ]:
from gensim.models import Word2Vec

# Charger un modèle Word2Vec pré-entraîné
model = Word2Vec.load("path_to_model")

# Accéder au vecteur d'un mot
word_vector = model.wv['happy']
print(word_vector)


In [15]:
#Ce matin 
import pandas as pd
import nltk

# Télécharger le modèle de tokenisation de phrases si besoin
nltk.download('punkt')

# Exemple de dataframe avec un commentaire par ligne
df = pd.DataFrame({
    'commentaires': [
        "Je trouve que le salaire est trop bas. J'apprécie beaucoup la flexibilité du télétravail. Concernant l'ambiance au bureau, je la trouve stressante.",
        
    "rémunération est logique. J'apprécie beaucoup les jours du télétravail. Concernant l'ambiance au travail, je la trouve parfaite."
    ]
})

# Tokenizer pour diviser en phrases
df['phrases'] = df['commentaires'].apply(nltk.sent_tokenize)

print(df['phrases'])


0    [Je trouve que le salaire est trop bas., J'app...
1    [rémunération est logique., J'apprécie beaucou...
Name: phrases, dtype: object


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\POSTE\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [16]:
# Listes de mots-clés par thématique
mots_qualite_vie = ['ambiance', 'conditions', 'environnement', 'stress', 'bien-être', 'collègues', 'bureau']
mots_equilibre = ['horaires', 'famille', 'équilibre', 'télétravail', 'flexibilité', 'temps libre']
mots_remuneration = ['salaire', 'prime', 'rémunération', 'augmentation', 'compensation', 'avantages']

# Fonction pour classifier chaque phrase
def classer_thematique(phrase):
    phrase = phrase.lower()
    if any(mot in phrase for mot in mots_qualite_vie):
        return 'Qualité de vie au travail'
    elif any(mot in phrase for mot in mots_equilibre):
        return 'Équilibre vie professionnelle/personnelle'
    elif any(mot in phrase for mot in mots_remuneration):
        return 'Rémunération'
    else:
        return 'Inconnu'

# Fonction pour traiter un commentaire et classifier ses phrases
def classifier_commentaire(commentaire):
    phrases = nltk.sent_tokenize(commentaire)
    classes = {'Qualité de vie au travail': '', 'Équilibre vie professionnelle/personnelle': '', 'Rémunération': ''}
    
    for phrase in phrases:
        thematique = classer_thematique(phrase)
        if thematique != 'Inconnu':
            classes[thematique] += phrase + ' '  # Ajouter la phrase à la bonne catégorie

    return classes

# Appliquer la fonction sur chaque commentaire
df['classifications'] = df['commentaires'].apply(classifier_commentaire)

# Extraire les phrases pour chaque thématique
df['qualite_vie'] = df['classifications'].apply(lambda x: x['Qualité de vie au travail'])
df['equilibre'] = df['classifications'].apply(lambda x: x['Équilibre vie professionnelle/personnelle'])
df['remuneration'] = df['classifications'].apply(lambda x: x['Rémunération'])

print(df[['qualite_vie', 'equilibre', 'remuneration']])


                                         qualite_vie  \
0  Concernant l'ambiance au bureau, je la trouve ...   
1  Concernant l'ambiance au travail, je la trouve...   

                                           equilibre  \
0  J'apprécie beaucoup la flexibilité du télétrav...   
1     J'apprécie beaucoup les jours du télétravail.    

                              remuneration  
0  Je trouve que le salaire est trop bas.   
1               rémunération est logique.   


In [20]:
df

,commentaires,phrases,classifications,qualite_vie,equilibre,remuneration
0,Je trouve que le salaire est trop bas. J'appré...,"[Je trouve que le salaire est trop bas., J'app...",{'Qualité de vie au travail': 'Concernant l'am...,"Concernant l'ambiance au bureau, je la trouve ...",J'apprécie beaucoup la flexibilité du télétrav...,Je trouve que le salaire est trop bas.
1,rémunération est logique. J'apprécie beaucoup ...,"[rémunération est logique., J'apprécie beaucou...",{'Qualité de vie au travail': 'Concernant l'am...,"Concernant l'ambiance au travail, je la trouve...",J'apprécie beaucoup les jours du télétravail.,rémunération est logique.


In [24]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from nltk.tokenize import word_tokenize

#Lemmatisation
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
lem=WordNetLemmatizer()

#Charger les stopwords
nltk.download('stopwords')
from nltk.corpus import stopwords
mots_vides=stopwords.words('english')
print('\n')
print(mots_vides)

# Exemple de dataframe avec des commentaires
df = pd.DataFrame({
    'commentaires': [
        "Le salaire est insuffisant pour la charge de travail.",
        "Je suis satisfait de mon équilibre entre vie personnelle et professionnelle.",
        "L'environnement de travail est très stressant.",
        "Je trouve les conditions de travail acceptables.",
        "La rémunération devrait être plus élevée pour le niveau de travail exigé."
    ]
})

# 1. Vectorisation des commentaires avec TF-IDF
vectorizer = TfidfVectorizer(stop_words='english')
X = vectorizer.fit_transform(df['commentaires'])

# 2. Appliquer KMeans pour regrouper les commentaires
n_clusters = 3  # Nombre de thématiques à identifier (qualité de vie, rémunération, équilibre)
kmeans = KMeans(n_clusters=n_clusters, random_state=42)
df['cluster'] = kmeans.fit_predict(X)

# 3. Analyse des clusters (manuellement)
print(df)




['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than',

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\POSTE\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\POSTE\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [25]:
from transformers import BertModel, BertTokenizer
import torch

# Charger le modèle et le tokenizer BERT
model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)

# Encoder les commentaires avec BERT
def encode_commentaire(commentaire):
    inputs = tokenizer(commentaire, return_tensors='pt', padding=True, truncation=True)
    outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).detach().numpy()

# Appliquer l'encodage BERT à tous les commentaires
df['embeddings'] = df['commentaires'].apply(encode_commentaire)


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

C:\Users\POSTE\AppData\Roaming\Python\Python39\site-packages\huggingface_hub\file_download.py:159: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\POSTE\.cache\huggingface\hub\models--bert-base-uncased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Pl

In [ ]:
from nltk.tokenize import word_tokenize

#Lemmatisation
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
lem=WordNetLemmatizer()

#Charger les stopwords
nltk.download('stopwords')
from nltk.corpus import stopwords
mots_vides=stopwords.words('english')
print('\n')
print(mots_vides)

#Nettoyage de tweet
#Pour manipuler des expressions regulieres
import re
#ponctuations
import string
ponctuations=list(string.punctuation)
print(ponctuations)

def clean_tweet(tweet,ponctuations,stopwords,lem):
    #Harmonisation de la classe
    temp=tweet.lower()
    #retirer les contractions en anglais,les mentions ,les @ ,#,de liens web
    temp=re.sub("'","",temp)
    
    #Retrait de la ponctuation
    temp="".join([char for char in list(temp) if not (char in ponctuations)])
    #Retrait des nombres
    
    temp=re.sub("[0-9]","",temp)
    
    #Tokenization
    
    temp=word_tokenize(temp)
    
    #Lemmatisation
    temp=[lem.lemmatize(mot) for mot in temp]
    #Retrai de stopwords
    
    temp=[mot for mot in temp if mot not in stopwords]
    #Retrait de tokens de moins de 3 caracteres
    
    temp=[mot for mot in temp if len(mot)>=3]
    #Reformer la chaine
    
    temp=" ".join(mot for mot in temp)
    
    return temp

In [28]:
from nltk.tokenize import word_tokenize

#Lemmatisation
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
lem=WordNetLemmatizer()

#Charger les stopwords
nltk.download('stopwords')
from nltk.corpus import stopwords
mots_vides=stopwords.words('french')
print('\n')
print(mots_vides)

#Nettoyage de tweet
#Pour manipuler des expressions regulieres
import re
#ponctuations
import string
ponctuations=list(string.punctuation)
print(ponctuations)



['au', 'aux', 'avec', 'ce', 'ces', 'dans', 'de', 'des', 'du', 'elle', 'en', 'et', 'eux', 'il', 'ils', 'je', 'la', 'le', 'les', 'leur', 'lui', 'ma', 'mais', 'me', 'même', 'mes', 'moi', 'mon', 'ne', 'nos', 'notre', 'nous', 'on', 'ou', 'par', 'pas', 'pour', 'qu', 'que', 'qui', 'sa', 'se', 'ses', 'son', 'sur', 'ta', 'te', 'tes', 'toi', 'ton', 'tu', 'un', 'une', 'vos', 'votre', 'vous', 'c', 'd', 'j', 'l', 'à', 'm', 'n', 's', 't', 'y', 'été', 'étée', 'étées', 'étés', 'étant', 'étante', 'étants', 'étantes', 'suis', 'es', 'est', 'sommes', 'êtes', 'sont', 'serai', 'seras', 'sera', 'serons', 'serez', 'seront', 'serais', 'serait', 'serions', 'seriez', 'seraient', 'étais', 'était', 'étions', 'étiez', 'étaient', 'fus', 'fut', 'fûmes', 'fûtes', 'furent', 'sois', 'soit', 'soyons', 'soyez', 'soient', 'fusse', 'fusses', 'fût', 'fussions', 'fussiez', 'fussent', 'ayant', 'ayante', 'ayantes', 'ayants', 'eu', 'eue', 'eues', 'eus', 'ai', 'as', 'avons', 'avez', 'ont', 'aurai', 'auras', 'aura', 'aurons', 'a

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\POSTE\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\POSTE\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [29]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('stopwords')
nltk.download('punkt')
stop_words = stopwords.words('english')

def clean_text(text):
    # Suppression des caractères spéciaux et de la ponctuation
    processed_tweet = re.sub(r"[^\w\s]", "", text)
    
    # Conversion en minuscules
    processed_tweet = processed_tweet.lower()
    
    # Tokenisation du texte
    tokens = word_tokenize(processed_tweet)
    
    # Suppression des mots vides
    stop_words = set(stopwords.words("english"))
    tokens = [word for word in tokens if word not in stop_words]
    
    # Lemmatisation des tokens
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    
    # Joindre les tokens en une seule chaîne
    cleaned_text = ' '.join(tokens)
    
    return cleaned_text
# La Lemmatisation


# Appliquer le nettoyage du texte sur la colonne 'texte'
df['texte_nettoye'] = df['text'].apply(clean_text)

df

['au',
 'aux',
 'avec',
 'ce',
 'ces',
 'dans',
 'de',
 'des',
 'du',
 'elle',
 'en',
 'et',
 'eux',
 'il',
 'ils',
 'je',
 'la',
 'le',
 'les',
 'leur',
 'lui',
 'ma',
 'mais',
 'me',
 'même',
 'mes',
 'moi',
 'mon',
 'ne',
 'nos',
 'notre',
 'nous',
 'on',
 'ou',
 'par',
 'pas',
 'pour',
 'qu',
 'que',
 'qui',
 'sa',
 'se',
 'ses',
 'son',
 'sur',
 'ta',
 'te',
 'tes',
 'toi',
 'ton',
 'tu',
 'un',
 'une',
 'vos',
 'votre',
 'vous',
 'c',
 'd',
 'j',
 'l',
 'à',
 'm',
 'n',
 's',
 't',
 'y',
 'été',
 'étée',
 'étées',
 'étés',
 'étant',
 'étante',
 'étants',
 'étantes',
 'suis',
 'es',
 'est',
 'sommes',
 'êtes',
 'sont',
 'serai',
 'seras',
 'sera',
 'serons',
 'serez',
 'seront',
 'serais',
 'serait',
 'serions',
 'seriez',
 'seraient',
 'étais',
 'était',
 'étions',
 'étiez',
 'étaient',
 'fus',
 'fut',
 'fûmes',
 'fûtes',
 'furent',
 'sois',
 'soit',
 'soyons',
 'soyez',
 'soient',
 'fusse',
 'fusses',
 'fût',
 'fussions',
 'fussiez',
 'fussent',
 'ayant',
 'ayante',
 'ayantes',


In [ ]:
# Fonction pour générer le word cloud
def generer_wordcloud(tweets, titre):
    tout_texte = ' '.join(tweets)
    wordcloud = WordCloud(width=800, height=400, background_color='white').generate(tout_texte)
    plt.figure(figsize=(10, 5))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.title(titre, fontsize=14)
    plt.axis('off')
    plt.show()

# Générer le word cloud 
generer_wordcloud(df['text'], 'Word Cloud - des tweets')

In [ ]:
from collections import Counter
# pour obtenir les mots les plus utilisés
def get_maxtoken(tweets, num=30):
  word_tokens = Counter(tweets)
  max_common = word_tokens.most_common(num)
  return dict(max_common)